In [1]:
import mne
import pandas as pd
import tools.helpers
import os 
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path

from config import Config
configObj = Config()
configss = configObj.getConfigSnapshot()

In [2]:
out = os.path.normpath("C:\Users\erang\Desktop\Reading_task\ov_experiment\scenarios\data\record_p2.gdf")

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (221486322.py, line 1)

In [2]:
# partipant_data_path = 'C:\Users\erang\Desktop\Reading_task\ov_experiment\scenarios\data\record_p-5_2024.06.10-16.08.57.gdf'
raw  = mne.io.read_raw_gdf('C:\\Users\\erang\\Desktop\\Reading_task\\ov_experiment\\scenarios\\data\\record_p1.gdf')

AttributeError: module 'mne' has no attribute 'io'

In [6]:
print(mne)

<module 'mne' (namespace)>


In [34]:
events_from_annot, event_dict =  mne.events_from_annotations(raw)

Used Annotations descriptions: ['32769', '32770', '33025', '33026', '33027', '33028', '33029', '33030']


In [74]:
stimcodes =  tools.helpers.getOVStimCodes()
stimGroups = tools.helpers.getStimGroups()

inv_stimCodesMap =  { v: k for k, v in stimcodes.items()}
inv_stimGroupsMap = { v: k for k, v in stimGroups.items() }

    # if check to make sure only get the relevant keys 
modified_event_dict =  { inv_stimGroupsMap[inv_stimCodesMap[int(k)]] :
                            v for k,v in event_dict.items() 
                            if inv_stimCodesMap[int(k)] in inv_stimGroupsMap}

timings ={}


timings['timing/distractive/start'] = [event[0] / raw.info['sfreq'] for event in events_from_annot if event[2] ==  modified_event_dict['timing/distractive/start']]
timings['timing/distractive/stop'] = [event[0] / raw.info['sfreq'] for event in events_from_annot if event[2] ==  modified_event_dict['timing/distractive/stop']]
timings['timing/attentive/start'] = [event[0] / raw.info['sfreq'] for event in events_from_annot if event[2] ==  modified_event_dict['timing/attentive/start']]
timings['timing/attentive/stop'] = [event[0] / raw.info['sfreq'] for event in events_from_annot if event[2] ==  modified_event_dict['timing/attentive/stop']]



In [77]:
raw_segment = raw.copy().crop(tmin=timings['timing/distractive/start'] [0] ,
                                  tmax=timings['timing/distractive/stop'] [0])

In [78]:
raw_segment

Measurement date,Unknown
Experimenter,Unknown
Participant,0x00000000
Digitized points,Not available
Good channels,66 EEG
Bad channels,None
EOG channels,Not available
ECG channels,Not available
Sampling frequency,500.00 Hz
Highpass,0.00 Hz
Lowpass,250.00 Hz


: 

In [72]:
events = [event[0]/ raw.info['sfreq'] for event in events_from_annot if event[2] ==  modified_event_dict['timing/distractive/start']]

In [73]:
events

[578.176, 1122.758]

In [59]:
ev = events_from_annot[1]

In [62]:
ev[0]

191675